# Example 1 : Regression / Interpolation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate

%matplotlib inline

In [ ]:
# Create Sample Datapoints
x = np.linspace(0, 10, 10)
y = np.exp(x)
plt.scatter(x, y)

In [ ]:
# Trying to predict the in-between points. (It's often use for 'upsampling')
x_pred = np.linspace(0, 10, 50)

# Do interpolation using linear kernel
s_linear = interpolate.interp1d(x, y) # kind='cubic'
y_linear = s_linear(x_pred)

# Do interpolation using cubic (polynomial) kernel
s_cubic = interpolate.interp1d(x, y, kind='cubic')
y_cubic = s_cubic(x_pred)

In [ ]:
# Compair the Model
plt.plot(x, y, 'x', x_pred, y_linear, 'b', x_pred, y_cubic, 'r')
plt.legend(['Input Data', 'Linear', 'Cubic'])
plt.title('Interpolation')
plt.show()

In [ ]:
# Interpolation and only done the prediction in the range of training.
# To predict the point which not in that range, use 'spline'.

x_spline = np.linspace(0, 12, 100)
s_spline = interpolate.splrep(x, y) # kind='cubic'
y_spline = interpolate.splev(x_spline, s_spline)

In [ ]:
plt.plot(x, y, 'x',x_spline, y_spline, 'g',  x_pred, y_cubic, 'r', )
plt.legend(['Input Data', 'Spline', 'Interpolation'])
plt.title('Interpolation')
plt.show()

In [ ]:
# 2D Spline also usefull for upsampling, especially in image processing.

x_edges, y_edges = np.mgrid[-1:1:21j, -1:1:21j]
x = x_edges[:-1, :-1] + np.diff(x_edges[:2, 0])[0] / 2.
y = y_edges[:-1, :-1] + np.diff(y_edges[0, :2])[0] / 2.
z = (x+y) * np.exp(-6.0*(x*x+y*y))

In [ ]:
plt.figure()
lims = dict(cmap='RdBu_r', vmin=-0.25, vmax=0.25)
plt.pcolormesh(x_edges, y_edges, z, shading='flat', **lims)
plt.colorbar()
plt.title("Sparsely sampled function.")
plt.show()

In [ ]:
xnew_edges, ynew_edges = np.mgrid[-1:1:71j, -1:1:71j]
xnew = xnew_edges[:-1, :-1] + np.diff(xnew_edges[:2, 0])[0] / 2.
ynew = ynew_edges[:-1, :-1] + np.diff(ynew_edges[0, :2])[0] / 2.
tck = interpolate.bisplrep(x, y, z, s=0)
znew = interpolate.bisplev(xnew[:,0], ynew[0,:], tck)

In [ ]:
plt.figure()
plt.pcolormesh(xnew_edges, ynew_edges, znew, shading='flat', **lims)
plt.colorbar()
plt.title("Interpolated function.")
plt.show()

# Example 2 : Power Spectrum Density

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.gridspec as gridspec

# Set some random seed
np.random.seed(12123)

# Common paremeters
fs = 128
dt = 1/fs
t = np.arange(0, 10, dt)
n = len(t)

# Generaten random noise
nse = np.random.randn(n)
r = np.exp(-t / 0.05)
cnse = np.convolve(nse, r) * dt
cnse = cnse[:n]

# Generate carrier frequence with random noise
s = 0.1 * np.sin(2 * np.pi * t) + cnse

# Plot Signal with PSD
fig, (ax0, ax1) = plt.subplots(2, 1)
ax0.plot(t, s)

# False = No Scale = Power Spectrum / True = Scale = Unit is dB/Hz = Power Spectral Density
p, freq = ax1.psd(s, n, fs, scale_by_freq=True) 

In [ ]:
# Export the psd array for using in the future.
plt.plot(freq, 10*np.log10(p)) # norm as DB

# Example 3 : Spectrogram

In [ ]:
# Create action_signal
action_signal = 0.05 * np.sin(25 * 2 * np.pi * t)
action_signal[0:300] = 0
action_signal[400:] = 0

# Generate S with action_signal
s_a = s + action_signal



fig, (ax0, ax1, ax2, ax3) = plt.subplots(4, 1)
ax0.plot(t, s)
ax1.plot(t, action_signal)
ax2.plot(t, s_a)
p, freq = ax3.psd(s_a, n, fs, scale_by_freq=True) 

In [ ]:
fig, (ax0, ax1, ax2, ax3) = plt.subplots(4, 1)
ax0.plot(t, s)
ax1.plot(t, action_signal)
ax2.plot(t, s_a)
p, freq, tout, img = ax3.specgram(s_a, fs, fs, scale_by_freq=True, noverlap=fs/2) 

# Example 4 : PCA

In [ ]:
# load IRIS dataset
import pandas as pd
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
df = pd.read_csv(url, names=['sepal length','sepal width','petal length','petal width','target'])

In [ ]:
from sklearn.preprocessing import StandardScaler
features = ['sepal length', 'sepal width', 'petal length', 'petal width']

# Separating out the features
x = df.loc[:, features].values

# Separating out the target
y = df.loc[:,['target']].values

# Standardizing the features
x = StandardScaler().fit_transform(x)

In [ ]:
from sklearn.decomposition import PCA

# Calculate the PCA
pca = PCA()
pca_model = pca.fit_transform(x)

In [ ]:
# show all eigenvalue
pca.explained_variance_ratio_

In [ ]:
# show all eigenvector
for eigenvector in pca.components_:
    print(eigenvector)

In [ ]:
# Calculate and pick only best n components
pca = PCA(n_components=2)
pca_model = pca.fit_transform(x)

pca_df = pd.DataFrame(data = pca_model, columns = ['PC 1', 'PC 2'])

In [ ]:
final_df = pd.concat([pca_df, df[['target']]], axis = 1)

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('PC 1', fontsize = 15)
ax.set_ylabel('PC 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
colors = ['r', 'g', 'b']
for target, color in zip(targets,colors):
    indicesToKeep = final_df ['target'] == target
    ax.scatter(final_df.loc[indicesToKeep, 'PC 1']
               , final_df.loc[indicesToKeep, 'PC 2']
               , c = color
               , s = 50)
ax.legend(targets)
ax.grid()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
for eigenvector in pca.components_:
    print(eigenvector)

# Example 5 : ICA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from sklearn.decomposition import FastICA, PCA

# Common parameters
np.random.seed(12123)
n = 1000
t = np.linspace(0, 8, n)


In [ ]:
# Generate Signal Sources
s1 = np.sin(np.pi * t)         # Signal 1 : Sin
s2 = np.sign(np.sin(5 * t))    # Signal 2 : Square
s3 = np.sin(3 * np.pi * t)     # Signal 3 : Sin (Higher Freq.)

S = np.c_[s1, s2, s3]

# Add Noise
S += 0.2 * np.random.normal(size=S.shape)

# Standardize data
S /= S.std(axis=0)  

In [ ]:
# Cretae a Mixing Matrix (Transfer function of source->measurement)
A = np.array([[1, 1, 1], [0.5, 3.0, 1.0], [2.0, 1.0, 3.0]])

# Generate the simulated observation
X = np.dot(S, A.T)

In [ ]:
# Compute ICA
ica = FastICA(n_components=3)
S_ = ica.fit_transform(X)

In [ ]:
# Show estimated transfer function (ideal: equal to mixing matrix)
A_ = ica.mixing_  
A_

In [ ]:
# Reconstruct data using reverse of estimated mixing matrix.
assert np.allclose(X, np.dot(S_, A_.T) + ica.mean_)

In [ ]:
# Show Result
plt.figure()

models = [X, S, S_]
names = [
    "Observations (mixed signal)",
    "True Sources",
    "ICA recovered signals"
]

colors = ["steelblue", "orange", "red"]

for ii, (model, name) in enumerate(zip(models, names), 1):
    plt.subplot(3, 1, ii)
    plt.title(name)
    for sig, color in zip(model.T, colors):
        plt.plot(sig, color=color)

plt.tight_layout()
plt.show()

# Example 6 : EEG Band Power

In [ ]:
# load dataset
#http://bnci-horizon-2020.eu/database/data-sets
# 2. Two class motor imagery (002-2014)

!wget http://bnci-horizon-2020.eu/database/data-sets/002-2014/S01T.mat

In [ ]:
# load .mat to python
import scipy.io
mat = scipy.io.loadmat('/content/S01T.mat')

mat.keys()

In [ ]:
# get only expected data ([0,0] must used due to the limitation of the library.)
session_data = mat['data'][0,0]

In [ ]:
# get a session of data
eeg = session_data['X'][0,0] # [:, ch]
marker = session_data['trial'][0,0]
fs = int(session_data['fs'][0,0])
class_id = session_data['y']

# convert to string label
class_label = list(session_data['classes'][0,0][0])
class_label = [str(i[0]) for i in class_label]
class_label

In [ ]:
# select on one channel
ch_id = 7

# select the range of data 
# For demo: first 20 seconds : Class 1 for 10 secs and Class 2 for 10 secs
rawdata = eeg[:,0]  

fig, (ax0, ax1) = plt.subplots(2, 1)
ax0.plot(rawdata)
p, freq, tout, img = ax1.specgram(rawdata, fs, fs, scale_by_freq=True, noverlap=fs/2) # False = No Scale = Power Spectrum / True = Scale = Unit is dB/Hz = Power Spectral Density

In [ ]:
p.shape

In [ ]:
freq

In [ ]:
# Generate Band Power matrix
bp = np.zeros((p.shape[1], 5)) # 5 bands

# Delta : 0.5-4 Hz
ft = np.where(np.logical_and(freq >= 0.5,freq < 4))[0]
bp[:,0] = np.sum(p[ft,:], 0)

# Theta : 4-7 Hz
ft = np.where(np.logical_and(freq >= 4,freq < 8))[0]
bp[:,1] = np.sum(p[ft,:], 0)

# Alpha : 8-12 Hz
ft = np.where(np.logical_and(freq >= 8,freq < 13))[0]
bp[:,2] = np.sum(p[ft,:], 0)

# Beta : 13-30 Hz
ft = np.where(np.logical_and(freq >= 13,freq < 30))[0]
bp[:,3] = np.sum(p[ft,:], 0)

# Beta : 31 - 100 Hz (* based on paper.)
ft = np.where(np.logical_and(freq >=30,freq < 100))[0]
bp[:,4] = np.sum(p[ft,:], 0)

In [ ]:
features = ['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']
bp.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
features = ['D', 'T', 'A', 'B', 'G']

x = StandardScaler().fit_transform(bp)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
principalComponents = pca.fit_transform(x)

pca.explained_variance_ratio_